# Day 2 — Data Acquisition, Cleaning & Preprocessing

**Duration:** 2 hours

**Objectives:**
- Learn common data sources and acquisition methods
- Practice identifying and fixing common data problems
- Preprocess features for ML

## 1. Why data cleaning matters

Most real data is messy. Expect to spend 60–80% of time cleaning and preparing data. Dirty data leads to poor models (GIGO).

## 2. Data sources & acquisition methods

- Files: CSV, Excel
- Databases: SQL
- APIs: REST, JSON
- Web scraping
- Sensors / IoT

We'll focus on file-based and in-memory examples for this course.

## 3. Common data issues

- Missing values
- Duplicates
- Wrong datatypes
- Outliers
- Inconsistent formatting (e.g., '50,000' vs 50000)

We will address techniques for handling these problems.

## 4. Loading data in pandas (demo)

We'll demonstrate loading CSV and previewing data. Here we use the Titanic dataset again as it's convenient and illustrative.

In [ ]:
import pandas as pd
import seaborn as sns

titanic = sns.load_dataset('titanic')
# create a copy to experiment on
df = titanic.copy()
# show first rows and info
print(df.head())
print('\nINFO:\n')
print(df.info())

## 5. Missing values — detection

Look for nulls using `isnull().sum()` and visual inspection. Decide when to drop vs impute.

In [ ]:
# Count missing values per column
print(df.isnull().sum())

## 6. Handling missing values — strategies

- Drop rows/columns
- Impute with mean/median/mode
- Forward/backward fill for time-series
- Model-based imputation (advanced)

We'll impute `age` with median and `embarked` with mode as a simple strategy.

In [ ]:
# Impute age with median and embarked with mode (inplace copy)
from sklearn.impute import SimpleImputer
import numpy as np

imputer_age = SimpleImputer(strategy='median')
df['age'] = imputer_age.fit_transform(df[['age']])

imputer_emb = SimpleImputer(strategy='most_frequent')
df['embarked'] = imputer_emb.fit_transform(df[['embarked']])

print(df[['age','embarked']].isnull().sum())

## 7. Duplicates

Use `drop_duplicates()` to remove exact duplicates. Be careful: near-duplicates may need domain logic.

In [ ]:
# Demonstrate duplicates (Titanic has none usually)
df.shape
# if duplicates existed we'd use:
# df = df.drop_duplicates()


## 8. Outliers — detection & handling

Common methods: boxplot visualization, IQR rule, z-score. We'll show IQR method for fare.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(x=df['fare'].dropna())
plt.title('Boxplot of Fare')
plt.show()

# IQR method
Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
print('fare range (IQR):', lower, upper)

# Count potential outliers
outliers = df[(df['fare'] < lower) | (df['fare'] > upper)]
print('Potential outliers count:', outliers.shape[0])

## 9. Categorical encoding

Convert categorical variables to numeric: Label Encoding vs One-Hot Encoding. Example: `sex` and `class`.

In [ ]:
# One-hot encode 'class' and label encode 'sex'
from sklearn.preprocessing import LabelEncoder

df_enc = df.copy()
le = LabelEncoder()
df_enc['sex_encoded'] = le.fit_transform(df_enc['sex'].astype(str))
# one-hot for 'class'
df_enc = pd.get_dummies(df_enc, columns=['class'], prefix='class')

print(df_enc[['sex','sex_encoded']].head())
print('\nOne-hot columns:', [c for c in df_enc.columns if c.startswith('class_')])

## 10. Scaling & Normalization

Why scale? Many ML algorithms are sensitive to feature scale (e.g., KNN, SVM). Methods: MinMaxScaler, StandardScaler.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
df_enc['fare_scaled'] = scaler.fit_transform(df_enc[['fare']].fillna(0))

print(df_enc[['fare','fare_scaled']].head())

## 11. Putting it together — simple preprocessing pipeline

Steps applied: imputation, encoding, scaling. This prepares data for modeling.

In [ ]:
# Create a simple feature matrix and label for modeling demo
features = ['age','fare_scaled','sex_encoded']
# drop rows with missing label
df_model = df_enc.dropna(subset=['survived'])
X = df_model[features]
y = df_model['survived'].astype(int)

print('X shape:', X.shape, 'y shape:', y.shape)
X.head()

## 12. Exercise (in-notebook)

1. Try a different imputation strategy for `age` (mean instead of median). Compare means before/after.
2. Create one additional visualization: survival vs fare (bin fare into groups and plot survival rate).

(Optional) Try scaling with StandardScaler.

In [ ]:
# Exercise starters
# 1) Mean imputation
import numpy as np

df_alt = titanic.copy()
df_alt['age'] = df_alt['age'].fillna(df_alt['age'].mean())
print('Mean age after imputation:', df_alt['age'].mean())

# 2) Survival vs fare bins
bins = [0,10,20,50,100,600]
labels = ['0-10','10-20','20-50','50-100','100+']
df_model['fare_bin'] = pd.cut(df_model['fare'].fillna(0), bins=bins, labels=labels)
print(df_model.groupby('fare_bin')['survived'].mean())

import matplotlib.pyplot as plt
survival_by_fare = df_model.groupby('fare_bin')['survived'].mean().reset_index()
sns.barplot(x='fare_bin', y='survived', data=survival_by_fare)
plt.title('Survival rate by Fare bin')
plt.xlabel('Fare bin')
plt.ylabel('Survival rate')
plt.show()

## 13. Wrap-up & Reading

Suggested reading: Wes McKinney (Ch.3-4). Tomorrow: Day 3 — Statistics & Probability.